In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import time

In [11]:
df = pd.read_csv('input/Surgical-deepnet.csv')
df.tail()

,bmi,Age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,baseline_osteoart,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
14630,18.79,14.1,1,0,1,0,0,0,0,0,...,-0.54,1,0,7.78,0,1,0,-0.16,1,1
14631,19.65,12.6,0,0,0,0,0,0,0,0,...,-1.42,4,0,8.40,6,1,0,-0.77,1,1
14632,14.84,12.6,1,0,0,0,0,0,0,0,...,0.65,0,0,13.25,3,3,0,0.99,1,1
14633,17.75,8.9,0,0,1,0,0,0,1,0,...,-0.50,0,1,8.30,5,0,0,0.17,1,1
14634,14.40,6.1,1,0,1,0,0,0,1,0,...,0.78,2,0,7.65,4,1,0,1.06,0,1


In [12]:
df.isnull().sum()

bmi                    0
Age                    0
asa_status             0
baseline_cancer        0
baseline_charlson      0
baseline_cvd           0
baseline_dementia      0
baseline_diabetes      0
baseline_digestive     0
baseline_osteoart      0
baseline_psych         0
baseline_pulmonary     0
ahrq_ccs               0
ccsComplicationRate    0
ccsMort30Rate          0
complication_rsi       0
dow                    0
gender                 0
hour                   0
month                  0
moonphase              0
mort30                 0
mortality_rsi          0
race                   0
complication           0
dtype: int64

In [13]:
X = df.drop('complication', axis=1).copy().values
Y = df['complication'].copy().values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=.2)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape



((11708, 24), (2927, 24), (11708,), (2927,))

In [18]:
class Perceptron:
    def __init__(self, learning_rate, input_length):
        self.learning_rate = learning_rate
        self.weights = np.random.rand(input_length)
        self.bias = np.random.rand(1)

    def activation(self, x, function):
        '''function parameters:
        * `sigmoid`
        * `relu`
        * `tanh`
        * `linear`
         '''
        if function == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif function == 'relu':
            return np.maximum(0,x)
        elif function == 'tanh':
            return np.tanh(x)
        elif function == 'linear':
            return x
        else:
            raise Exception('Unknown Actication function')

    def fit(self, X_train, Y_train, epochs):
        for epoch in tqdm(range(epochs)):
            for x,y in zip(X_train, Y_train): 
                y_pred = x @ self.weights + self.bias
                y_pred = self.activation(y_pred,'sigmoid')

                error = y - y_pred

                self.weights += self.learning_rate * error * x
                self.bias += self.learning_rate * error


    def predict(self, X_test):
        Y_pred = []
        for x_test in X_test:
            y_pred = x_test @ self.weights + self.bias
            y_pred = self.activation(y_pred, 'sigmoid')
            Y_pred.append(y_pred)
        return np.array(Y_pred)


    def calc_loss(self, X_test, Y_test,metric):
        '''choose between: `mse` `mae` `rmse`'''
        Y_pred = self.predict(X_test)
        if metric == 'mse':
            return np.mean(np.square(Y_test - Y_pred))
        elif metric == 'mae':
            return np.mean(np.abs(Y_test - Y_pred))
        elif metric == 'rmse':
            return np.sqrt(np.mean(np.square(Y_test - Y_pred)))
        else:
            raise Exception('Unknown metric')
        
    def calc_accuracy(self, X_test, Y_test):
        Y_pred = self.predict(X_test)
        Y_pred = Y_pred.reshape(-1)
        Y_pred = np.where(Y_pred > .5, 1, 0)
        accuracy = np.sum(Y_pred == Y_test) / len(Y_test)
        return accuracy

    def evaluate(self, X_test, Y_test):
        loss = self.calc_loss(X_test, Y_test, 'mse')
        accuracy = self.calc_accuracy(X_test, Y_test)
        return loss, accuracy




In [19]:
model = Perceptron(learning_rate=.001,input_length=X_train.shape[1])
model.fit(X_train, Y_train, epochs=256)

100%|██████████| 256/256 [00:20<00:00, 12.32it/s]


In [20]:
model.evaluate(X_test,Y_test)

(0.39360747812321395, 0.598223436966177)

In [22]:
Y_pred = model.predict(X_test)
Y_pred = Y_pred.reshape(-1)
Y_pred = np.where(Y_pred > .5, 1, 0)

In [23]:
from sklearn.metrics import confusion_matrix

confusion_matrix(Y_test, Y_pred)

array([[1097, 1056],
       [ 120,  654]], dtype=int64)